In [1]:
#TODO: reduce size vocabulary
#TODO: check size embedding
#TODO: tensorboard

In [1]:
import pandas as pd

df = pd.read_csv('data/isear_processed.csv',header=0, index_col=0, names=['text','sentiment'])

In [2]:
df['sentiment'].unique()

array(['joy', 'fear', 'anger', 'sadness', 'disgust', 'shame', 'guilt'],
      dtype=object)

In [3]:
df.head()

,text,sentiment
0,"During the period of falling in love, each tim...",joy
1,When I was involved in a traffic accident.,fear
2,When I was driving home after several days of...,anger
3,When I lost the person who meant the most to me.,sadness
4,The time I knocked a deer down - the sight of ...,disgust


In [4]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
sequences = pad_sequences(sequences, maxlen=100)

/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/arnaudstiegler/miniconda3/envs/emotion_detection/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16"

In [5]:
import numpy as np

emotion_dict = {}
labels = df['sentiment']

for k in range(len(labels.unique())):
    emotion_dict[labels.unique()[k]] = k
    
encoded_labels = np.array(list(labels.map(lambda x: emotion_dict[x])))

In [6]:
embeddings_index = {}
with open('data/embedding/glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [7]:
#Preparing the embedding

MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

num_words = min(MAX_NUM_WORDS, len(tokenizer.word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in tokenizer.word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector


In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(sequences, encoded_labels, stratify=encoded_labels)

In [9]:
import tensorflow as tf

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(1024).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((X_test,y_test)).shuffle(1024).batch(32)

In [15]:
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.initializers import Constant

model = tf.keras.Sequential()

model.add(Embedding(num_words, EMBEDDING_DIM, embeddings_initializer=Constant(embedding_matrix),
                            input_length=100,
                            trainable=False))
model.add(LSTM(50, dropout=0.2, return_sequences=True))
model.add(LSTM(50, dropout=0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(rate=0.3))
model.add(Dense(7, activation='softmax'))

model.compile(optimizer="adam", loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 100, 100)          906400    
_________________________________________________________________
unified_lstm_2 (UnifiedLSTM) (None, 100, 50)           30200     
_________________________________________________________________
unified_lstm_3 (UnifiedLSTM) (None, 50)                20200     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1632      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 7)                 231       
Total params: 958,663
Trainable params: 52,263
Non-trainable params: 906,400
___________________________________________

In [16]:
import tensorflow as tf
assert tf.__version__ >= "2.0"

In [17]:
%load_ext tensorboard.notebook
!rm -rf ./tensorboard-logs/ # Clear any logs from previous runs

import datetime 
import os

date = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = os.path.join("./tensorboard-logs/", date)
print("Writing logs to", log_dir)
exp_dir = os.path.join(log_dir, "softmax")

tb_callback = tf.keras.callbacks.TensorBoard(log_dir=exp_dir)

The tensorboard.notebook extension is already loaded. To reload it, use:
  %reload_ext tensorboard.notebook
Writing logs to ./tensorboard-logs/20191031-143437


In [18]:
model.fit(X_train, y_train, 
          epochs = 30, 
          validation_data=(X_test,y_test),
         callbacks=[tb_callback])

Train on 5749 samples, validate on 1917 samples
Epoch 1/30
5749/5749 [==============================] - 27s 5ms/sample - loss: 1.8856 - accuracy: 0.2260 - val_loss: 1.7633 - val_accuracy: 0.3177
Epoch 2/30
5749/5749 [==============================] - 22s 4ms/sample - loss: 1.7124 - accuracy: 0.3286 - val_loss: 1.6528 - val_accuracy: 0.3568
Epoch 3/30
5749/5749 [==============================] - 22s 4ms/sample - loss: 1.5867 - accuracy: 0.3952 - val_loss: 1.5039 - val_accuracy: 0.4116
Epoch 4/30
5749/5749 [==============================] - 22s 4ms/sample - loss: 1.5098 - accuracy: 0.4399 - val_loss: 1.4651 - val_accuracy: 0.4382
Epoch 5/30
5749/5749 [==============================] - 22s 4ms/sample - loss: 1.4626 - accuracy: 0.4563 - val_loss: 1.3805 - val_accuracy: 0.4862
Epoch 6/30
5749/5749 [==============================] - 22s 4ms/sample - loss: 1.3929 - accuracy: 0.4923 - val_loss: 1.3472 - val_accuracy: 0.5055
Epoch 7/30
5749/5749 [==============================] - 22s 4ms/sample

In [19]:
%tensorboard --logdir "$log_dir"